# Twitter Scraper

In [1]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet    

# application variables
user = 'israel.dryer@gmail.com'
search_term = '#polynote'
my_password = getpass()

# create instance of web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys(user)

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(1)

# find search input and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)

# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

 ·············


In [2]:
data[:2]

[('Websystemer',
  '@Websystemer',
  '2020-09-20T06:23:53.000Z',
  'Polynote by Netflix\u200a—\u200aA Unified Notebook for Writing Polyglot Code - https://websystemer.no/polynote-by-netflix%e2%80%8a-%e2%80%8aa-unified-notebook-for-writing-polyglot-code/…\n\n #machinelearning #multiplelanguagesupport #polynote #ravishankar #unifiednotebook',
  '',
  '',
  ''),
 ('TheSequenceAI',
  '@TheSequenceAI',
  '2020-09-03T14:49:00.000Z',
  "-What’s the main limitation of data science notebooks in machine learning architectures?\n-What is the core capability of Netflix’s #Polynote?\n#QuizTime #datascience #machinelearning #programming #ArtificialIntelligence #netflix\nOur regular quiz is here:TheSequence Quiz\nWelcome to The Sequence Quiz. Are you ready to test your knowledge after reading TheSequence Edge#18? Be sure to be subscribed to TheSequence.ai if you haven't already: for the active participants we...\ndocs.google.com",
  '',
  '3',
  '2')]

## Saving the tweet data

In [3]:
with open('polynote_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)